# **Lab1: Regression**
In *lab 1*, you need to finish:

1.  Basic Part: Implement the regression model to predict people's grip force from their weight.
You can use either Matrix Inversion or Gradient Descent.


> *   Step 1: Split Data
> *   Step 2: Preprocess Data
> *   Step 3: Implement Regression
> *   Step 4: Make Prediction
> *   Step 5: Train Model and Generate Result

2.  Advanced Part: Implementing a regression model to predict grip force in a different way (for example, with more variables) than the basic part




---
# 1. Basic Part (50%)
In the first part, you need to implement the regression to predict grip force

Please save the prediction result in a CSV file and submit it to Kaggle

### Import Packages

> Note: You **cannot** import any other package


In [73]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import csv
import math
import random

### Global attributes
Define the global attributes\
You can also add your own global attributes here

In [74]:
training_dataroot = 'lab1_basic_training.csv' # Training data file file named as 'lab1_basic_training.csv'
testing_dataroot = 'lab1_basic_testing.csv'   # Testing data file named as 'lab1_basic_testing.csv'
output_dataroot = 'lab1_basic.csv' # Output file will be named as 'lab1_basic.csv'

training_datalist =  [] # Training datalist, saved as numpy array
testing_datalist =  [] # Testing datalist, saved as numpy array

output_datalist =  [] # Your prediction, should be a list with 100 elements

### Load the Input File
First, load the basic input file **lab1_basic_training.csv** and **lab1_basic_testing.csv**

Input data would be stored in *training_datalist* and *testing_datalist*

In [75]:
# Read input csv to datalist
with open(training_dataroot, newline='') as csvfile:
  training_datalist = pd.read_csv(training_dataroot).to_numpy()

with open(testing_dataroot, newline='') as csvfile:
  testing_datalist = pd.read_csv(testing_dataroot).to_numpy()

### Implement the Regression Model

> Note: It is recommended to use the functions we defined, you can also define your own functions

#### Step 1: Split Data
Split data in *training_datalist* into training dataset and validation dataset


In [76]:
def SplitData(data, split_ratio):
    """
    Splits the given dataset into training and validation sets based on the specified split ratio.

    Parameters:
    - data (numpy.ndarray): The dataset to be split. It is expected to be a 2D array where each row represents a data point and each column represents a feature.
    - split_ratio (float): The ratio of the data to be used for training. For example, a value of 0.8 means 80% of the data will be used for training and the remaining 20% for validation.

    Returns:
    - training_data (numpy.ndarray): The portion of the dataset used for training.
    - validation_data (numpy.ndarray): The portion of the dataset used for validation.

    """
    training_data = []
    validation_data = []

    # TODO
    data_length = len(data)
    training_data_length = math.floor(data_length * split_ratio)
    random_index = random.sample(range(0, data_length), data_length)
    training_data_index = random_index[:training_data_length]
    validation_data_index = random_index[training_data_length + 1:]
    for idx in training_data_index:
        training_data.append(data[idx])
    for idx in validation_data_index:
        validation_data.append(data[idx])
        
    training_data = np.array(training_data)
    validation_data = np.array(validation_data)

    return training_data, validation_data



#### Step 2: Preprocess Data
Handle unreasonable data and missing data

> Hint 1: Outliers and missing data can be addressed by either removing them or replacing them using statistical methods (e.g., the mean of all data).

> Hint 2: Missing data are represented as `np.nan`, so functions like `np.isnan()` can be used to detect them.

> Hint 3: Methods such as the Interquartile Range (IQR) can help detect outliers

In [77]:
def PreprocessData(data):
    """
    Preprocess the given dataset and return the result.

    Parameters:
    - data (numpy.ndarray): The dataset to preprocess. It is expected to be a 2D array where each row represents a data point and each column represents a feature.

    Returns:
    - preprocessedData (numpy.ndarray): Preprocessed data.
    """
    preprocessedData = []

    # TODO
    noNanData = []
    for row in range(len(data)):
        if not (np.isnan(data[row][0]) or np.isnan(data[row][1])):
            noNanData.append(data[row])
    
    noNanData = np.array(noNanData)
    
    weight_data = noNanData[:, 0]
    grip_force_data = noNanData[:, -1]
    
    weight_q1 = np.percentile(weight_data, 25)
    weight_q3 = np.percentile(weight_data, 75)
    weight_qd = weight_q3 - weight_q1
    grip_force_q1 = np.percentile(grip_force_data, 25)
    grip_force_q3 = np.percentile(grip_force_data, 75)
    grip_force_qd = grip_force_q3 - grip_force_q1
    
    weight_upper_bound = weight_q3 + 1.5 * weight_qd
    weight_lower_bound = weight_q1 - 1.5 * weight_qd
    grip_force_upper_bound = grip_force_q3 + 1.5 * grip_force_qd
    grip_force_lower_bound = grip_force_q1 - 1.5 * grip_force_qd
    
    weight_mean = np.mean(weight_data)
    grip_force_mean = np.mean(grip_force_data)
    
    for row in noNanData:
        weight_out_of_bound = row[0] > weight_upper_bound or row[0] < weight_lower_bound
        grip_force_out_of_bound = row[1] > grip_force_upper_bound or row[1] < grip_force_lower_bound
        is_error_data = weight_out_of_bound or grip_force_out_of_bound
                
        if (not is_error_data):
            preprocessedData.append(row)
        else:
            preprocessedData.append([weight_mean, grip_force_mean])
            
    preprocessedData = np.array(preprocessedData)

    return preprocessedData


### Step 3: Implement Regression
You have to use Gradient Descent to finish this part

In [78]:
def Regression(dataset):
    """
    Performs regression on the given dataset and return the coefficients.

    Parameters:
    - dataset (numpy.ndarray): A 2D array where each row represents a data point.

    Returns:
    - w (numpy.ndarray): The coefficients of the regression model. For example, y = w[0] + w[1] * x + w[2] * x^2 + ...
    """

    X = dataset[:, :1]
    y = dataset[:, 1]

    # TODO: Decide on the degree of the polynomial
    degree = 2  # For example, quadratic regression

    # Add polynomial features to X
    X_poly = np.ones((X.shape[0], 1))  # Add intercept term (column of ones)
    for d in range(1, degree + 1):
        X_poly = np.hstack((X_poly, X ** d))  # Add x^d terms to feature matrix

    # Initialize coefficients (weights) to zero
    num_dimensions = X_poly.shape[1]  # Number of features (including intercept and polynomial terms)
    w = np.zeros(num_dimensions)

    # TODO: Set hyperparameters
    num_iteration = 1000
    learning_rate = 0.000000000001

    # Gradient Descent
    m = len(y)  # Number of data points
    g = np.zeros(num_dimensions)
    for iteration in range(num_iteration):
        # TODO: Prediction using current weights and compute error
        predicted_y = X_poly.dot(w)
        err = y - predicted_y

        # TODO: Compute gradient
        g = -2 * X_poly.T.dot(err)

        # TODO: Update the weights
        w = w - learning_rate * g
        print(w)
        

        # TODO: Optionally, print the cost every 100 iterations
        if iteration % 100 == 0:
            cost = ...
            print(f"Iteration {iteration}, Cost: {cost}")
            
            
    # print(f"eoko = {w}")
    return w


### Step 4: Make Prediction
Make prediction of testing dataset and store the value in *output_datalist*

In [79]:

def MakePrediction(w, test_dataset):
    """
    Predicts the output for a given test dataset using a regression model.

    Parameters:
    - w (numpy.ndarray): The coefficients of the model, where each element corresponds to
                               a coefficient for the respective power of the independent variable.
    - test_dataset (numpy.ndarray): A 1D array containing the input values (independent variable)
                                          for which predictions are to be made.

    Returns:
    - list/numpy.ndarray: A list or 1d array of predicted values corresponding to each input value in the test dataset.
    """
    prediction = []

    # TODO
    for idx in range(len(test_dataset)):
        predicted_y = 0
        for dim in range(len(w)):
            predicted_y += w[dim] * (test_dataset[idx][0] ** dim)
        # print(f"kkk = {test_dataset[idx][0] ** dim}")
        prediction.append(predicted_y)
        
    prediction = np.array(prediction)
    # print(f"pre = {w}")

    return prediction


### Step 5: Train Model and Generate Result

Use the above functions to train your model on training dataset, and predict the answer of testing dataset.

Save your predicted values in `output_datalist`

> Notice: **Remember to inclue the coefficients of your model in the report**



In [80]:
# TODO

# (1) Split data
training_dataset, validation_dataset = SplitData(training_datalist, 0.8)

# (2) Preprocess data
training_dataset = PreprocessData(training_dataset)
validation_dataset = PreprocessData(validation_dataset)

# (3) Train regression model
w = Regression(training_dataset)

# (4) Predict validation dataset's answer, calculate MAPE comparing to the ground truth
validation_predict_datalist = MakePrediction(w, validation_dataset)
mape = 0
for i in range(len(validation_dataset)):
    mape += abs((validation_dataset[i][-1] - validation_predict_datalist[i]) / validation_dataset[i][-1]) / len(validation_dataset)

print(f"mape = {mape}")

# (5) Make prediction of testing dataset and store the values in output_datalist
output_datalist = MakePrediction(w, testing_datalist)


[7.10785563e-07 4.94212066e-05 3.51707173e-03]
Iteration 0, Cost: Ellipsis
[1.16090571e-06 8.03378401e-05 5.68847810e-03]
[1.45009310e-06 9.98298977e-05 7.02908144e-03]
[1.63992187e-06 1.12268513e-04 7.85675365e-03]
[1.76840740e-06 1.20352388e-04 8.36774628e-03]
[1.85902009e-06 1.25747682e-04 8.68322347e-03]
[1.92625037e-06 1.29483066e-04 8.87799101e-03]
[1.97904451e-06 1.32193633e-04 8.99823336e-03]
[2.02292588e-06 1.34271488e-04 9.07246446e-03]
[2.06130455e-06 1.35958709e-04 9.11828859e-03]
[2.09628588e-06 1.37404756e-04 9.14657447e-03]
[2.12916970e-06 1.38701902e-04 9.16403239e-03]
[2.16075851e-06 1.39907119e-04 9.17480520e-03]
[2.19154777e-06 1.41055578e-04 9.18145069e-03]
[2.22184338e-06 1.42168994e-04 9.18554800e-03]
[2.25183418e-06 1.43260774e-04 9.18807208e-03]
[2.28163678e-06 1.44339196e-04 9.18962487e-03]
[2.31132317e-06 1.45409371e-04 9.19057798e-03]
[2.34093779e-06 1.46474452e-04 9.19116086e-03]
[2.37050807e-06 1.47536388e-04 9.19151517e-03]
[2.40005097e-06 1.48596381e-04 9

### *Write the Output File*

Write the prediction to output csv and upload the file to Kaggle
> Format: 'Id', 'gripForce'


In [81]:
# Assume that output_datalist is a list (or 1d array) with length = 100

with open(output_dataroot, 'w', newline='', encoding="utf-8") as csvfile:
  writer = csv.writer(csvfile)
  writer.writerow(['Id', 'gripForce'])
  for i in range(len(output_datalist)):
    writer.writerow([i,output_datalist[i]])


# 2. Advanced Part (45%)
In the second part, you need to implement regression differently from the basic part to improve your grip force predictions. You must use more than two features.

You can choose either matrix inversion or gradient descent for this part

We have provided `lab1_advanced_training.csv` for your training

> Notice: Be cautious of the "gender" attribute, as it is represented by "F"/"M" rather than a numerical value.

Please save the prediction result in a CSV file and submit it to Kaggle

In [82]:
training_dataroot = 'lab1_advanced_training.csv' # Training data file file named as 'lab1_advanced_training.csv'
testing_dataroot = 'lab1_advanced_testing.csv'   # Testing data file named as 'lab1_advanced_testing.csv'
output_dataroot = 'lab1_advanced.csv' # Output file will be named as 'lab1_advanced.csv'

training_datalist =  [] # Training datalist, saved as numpy array
testing_datalist =  [] # Testing datalist, saved as numpy array

output_datalist =  [] # Your prediction, should be a list with 3000 elements

In [83]:
# Read input csv to datalist
with open(training_dataroot, newline='') as csvfile:
  training_datalist = pd.read_csv(training_dataroot).to_numpy()

with open(testing_dataroot, newline='') as csvfile:
  testing_datalist = pd.read_csv(testing_dataroot).to_numpy()

In [84]:
# TODO

# (1) Split data

# (2) Preprocess data

# (3) Train regression model

# (4) Predict validation dataset's answer, calculate MAPE comparing to the ground truth

# (5) Make prediction of testing dataset and store the values in output_datalist

# Save the Code File
Please save your code and submit it as an ipynb file! (**Lab1.ipynb**)